In [1]:
import webbrowser, os
import json
import boto3
import io
from io import BytesIO
import sys
from pprint import pprint
import pdf2image
import os

In [2]:
def extract_jpg(file_name,csv_output_file):
    table_csv = get_table_csv_results(file_name)
    with open(csv_output_file, "w") as fout:
        fout.write(table_csv)
    
    print('CSV OUTPUT FILE: ',csv_output_file)
    
#extract_jpg('out.jpg')

In [3]:
def extract_pdf(file_name,csv_output_file):    
    from pdf2image import convert_from_path
    pages = convert_from_path(file_name, 500)
    table_csv=""
    for page in pages:
        page.save('out.jpeg', 'JPEG')
        table_csv+= get_table_csv_results('out.jpeg')
        os.remove("out.jpeg")
         
    with open(csv_output_file, "w") as fout:
        fout.write(table_csv)
        
    print('CSV OUTPUT FILE: ',csv_output_file)
    
        
#extract_pdf(file_name)    
    

In [4]:
def get_rows_columns_map(table_result, blocks_map):
    rows = {}
    for relationship in table_result['Relationships']:
        if relationship['Type'] == 'CHILD':
            for child_id in relationship['Ids']:
                cell = blocks_map[child_id]
                if cell['BlockType'] == 'CELL':
                    row_index = cell['RowIndex']
                    col_index = cell['ColumnIndex']
                    if row_index not in rows:
                        # create new row
                        rows[row_index] = {}
                        
                    # get the text value
                    rows[row_index][col_index] = get_text(cell, blocks_map)
    return rows


def get_text(result, blocks_map):
    text = ''
    if 'Relationships' in result:
        for relationship in result['Relationships']:
            if relationship['Type'] == 'CHILD':
                for child_id in relationship['Ids']:
                    word = blocks_map[child_id]
                    if word['BlockType'] == 'WORD':
                        text += word['Text'] + ' '
                    if word['BlockType'] == 'SELECTION_ELEMENT':
                        if word['SelectionStatus'] =='SELECTED':
                            text +=  'X '    
    return text


def get_table_csv_results(file_name):

    with open(file_name, 'rb') as file:
        img_test = file.read()
        bytes_test = bytearray(img_test)
        print('Image loaded', file_name)

    # process using image bytes
    # get the results
    client = boto3.client('textract')

    response = client.analyze_document(Document={'Bytes': bytes_test}, FeatureTypes=['TABLES'])

    # Get the text blocks
    blocks=response['Blocks']
    pprint(blocks)

    blocks_map = {}
    table_blocks = []
    for block in blocks:
        blocks_map[block['Id']] = block
        if block['BlockType'] == "TABLE":
            table_blocks.append(block)

    if len(table_blocks) <= 0:
        return "<b> NO Table FOUND </b>"

    csv = ''
    for index, table in enumerate(table_blocks):
        csv += generate_table_csv(table, blocks_map, index +1)
        csv += '\n\n'

    return csv

def generate_table_csv(table_result, blocks_map, table_index):
    rows = get_rows_columns_map(table_result, blocks_map)

    table_id = 'Table_' + str(table_index)
    
    # get cells.
    csv = 'Table: {0}\n\n'.format(table_id)

    for row_index, cols in rows.items():
        
        for col_index, text in cols.items():
            csv += '{}'.format(text) + ","
        csv += '\n'
        
    csv += '\n\n\n'
    return csv

In [8]:
file_name = 'sample form for grant of site clearance.pdf'
out= file_name[::-1]
output_file_name= out[out.index('.')+1:][::-1]
#output_file = file_name[:file_name.index('.')] + '.csv'

In [9]:
csv_output_file = output_file_name+'.csv'
if(file_name[-3:]=="pdf"):
    extract_pdf(file_name,csv_output_file)
else:
    extract_jpg(file_name,csv_output_file)

Image loaded out.jpeg
[{'BlockType': 'PAGE',
  'Geometry': {'BoundingBox': {'Height': 1.0,
                               'Left': 0.0,
                               'Top': 0.0,
                               'Width': 1.0},
               'Polygon': [{'X': 0.0, 'Y': 0.0},
                           {'X': 1.0, 'Y': 0.0},
                           {'X': 1.0, 'Y': 1.0},
                           {'X': 0.0, 'Y': 1.0}]},
  'Id': '73e827cc-0814-457c-9fc7-32973083b6a9',
  'Relationships': [{'Ids': ['52603c4e-561e-4a63-ab5d-68891d1827e8',
                             '9af96fff-e499-4a90-bb09-da3c76e8bad6',
                             '76eea17e-6fde-41aa-ba4f-ee0c02e24748',
                             '6670beb7-7c80-4c1c-9b39-783c4b05b810',
                             'a9233e71-ed6e-46e0-934e-2f00caee8089',
                             'd07b5b6a-5403-4157-9b83-a582e381b699',
                             '45e1e5bd-7357-47d5-bbf4-5d3fdd52cef9',
                             'e34befa1-58e8-40e

# EXTRACTING KEY VALUE PAIRS/JSON FILE OUT OF CSV FILE

In [11]:
import csv
file_name= csv_output_file
rows=[]
with open(file_name, 'r',encoding="utf8") as csvfile:
    csvreader = csv.reader(csvfile) 
      
    # extracting field names through first row 
    fields = next(csvreader) 
  
    # extracting each data row one by one 
    for row in csvreader: 
        rows.append(row) 
print(rows)

[[], ['Name of the Applicant ', 'Suresh Chaudhary ', '', ''], ['Tel. No. ', '9876543210 ', '', ''], ['Email Address ', 'sureshchaudhary@indiago.com ', '', ''], ['Name of nearest civilian airport ', 'Lucknow ', '', ''], ['Nearest Civilain airport distance from proposed airport ', '500kms ', '', ''], ['Ownership of Land ', 'Government ', 'Non-Agriculture ', ''], ['Public or Private airport ', 'Private ', '', ''], ['Exclusive cargo airport ', 'No ', '', ''], ['Wheather project located within 150kms of an existing civilian airport ', 'No ', '', ''], ['Ministry of Defense Approval Granted ', 'Yes ', '', ''], ['Ministry of Defense Comments ', 'None ', '', ''], ['Ministry of Home Affairs Approval Granted ', 'Yes ', '', ''], ['Ministry of Home Affairs Comments ', 'Level of security for nearby residents ', 'Moderate ', ''], [], [], [], [], [], ['Table: Table_1'], [], ['CBEC Approval Granted ', 'Yes ', '', ''], ['CBEC Comments Given ', 'None ', '', ''], ['Any other appovals ', 'No ', '', ''], ['

In [12]:
#removing empty lines
master_list = []
for i in range(len(rows)):  
    if(len(rows[i])>=2):
        master_list.append(rows[i])
print(master_list)

[['Name of the Applicant ', 'Suresh Chaudhary ', '', ''], ['Tel. No. ', '9876543210 ', '', ''], ['Email Address ', 'sureshchaudhary@indiago.com ', '', ''], ['Name of nearest civilian airport ', 'Lucknow ', '', ''], ['Nearest Civilain airport distance from proposed airport ', '500kms ', '', ''], ['Ownership of Land ', 'Government ', 'Non-Agriculture ', ''], ['Public or Private airport ', 'Private ', '', ''], ['Exclusive cargo airport ', 'No ', '', ''], ['Wheather project located within 150kms of an existing civilian airport ', 'No ', '', ''], ['Ministry of Defense Approval Granted ', 'Yes ', '', ''], ['Ministry of Defense Comments ', 'None ', '', ''], ['Ministry of Home Affairs Approval Granted ', 'Yes ', '', ''], ['Ministry of Home Affairs Comments ', 'Level of security for nearby residents ', 'Moderate ', ''], ['CBEC Approval Granted ', 'Yes ', '', ''], ['CBEC Comments Given ', 'None ', '', ''], ['Any other appovals ', 'No ', '', ''], ['Total Land area indentified ', '5', '000 hectare

In [13]:
master_dict = {}
for i in master_list:
    #try:
        master_dict[i[0]] = i[1:] 
    #except:
    #    print("error")
    #    print(rows.index(i))
print(master_dict.keys()) 

dict_keys(['Name of the Applicant ', 'Tel. No. ', 'Email Address ', 'Name of nearest civilian airport ', 'Nearest Civilain airport distance from proposed airport ', 'Ownership of Land ', 'Public or Private airport ', 'Exclusive cargo airport ', 'Wheather project located within 150kms of an existing civilian airport ', 'Ministry of Defense Approval Granted ', 'Ministry of Defense Comments ', 'Ministry of Home Affairs Approval Granted ', 'Ministry of Home Affairs Comments ', 'CBEC Approval Granted ', 'CBEC Comments Given ', 'Any other appovals ', 'Total Land area indentified ', 'Total cost of Land '])


In [14]:
import json
json_object = json.dumps(master_dict, indent = 4)   
print(json_object) 

{
    "Name of the Applicant ": [
        "Suresh Chaudhary ",
        "",
        ""
    ],
    "Tel. No. ": [
        "9876543210 ",
        "",
        ""
    ],
    "Email Address ": [
        "sureshchaudhary@indiago.com ",
        "",
        ""
    ],
    "Name of nearest civilian airport ": [
        "Lucknow ",
        "",
        ""
    ],
    "Nearest Civilain airport distance from proposed airport ": [
        "500kms ",
        "",
        ""
    ],
    "Ownership of Land ": [
        "Government ",
        "Non-Agriculture ",
        ""
    ],
    "Public or Private airport ": [
        "Private ",
        "",
        ""
    ],
    "Exclusive cargo airport ": [
        "No ",
        "",
        ""
    ],
    "Wheather project located within 150kms of an existing civilian airport ": [
        "No ",
        "",
        ""
    ],
    "Ministry of Defense Approval Granted ": [
        "Yes ",
        "",
        ""
    ],
    "Ministry of Defense Comments ": [
        "None

In [16]:
json_output_file = output_file_name+'.json'
with open(json_output_file,'w') as outfile:
    json.dump(json_object, outfile)    

In [ ]:
os.